## Mount & install

In [1]:
'''
function ClickConnect(){
    console.log("코랩 연결 끊김 방지");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect, 60 * 1000)
'''

'\nfunction ClickConnect(){\n    console.log("코랩 연결 끊김 방지");\n    document.querySelector("colab-toolbar-button#connect").click()\n}\nsetInterval(ClickConnect, 60 * 1000)\n'

In [2]:
!nvidia-smi

Thu Jul  6 01:44:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q timm
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [15]:
# 평가용 데이터 다운로드 (4분)
import urllib
import time
import os

start = time.time()
url = 'https://share.aiconnect.kr/fake_or_real/testFile/test.zip'
if not os.path.exists('test.zip'):
    urllib.request.urlretrieve(url, 'test.zip')
print('test time : ', time.strftime('%M:%S', time.gmtime(time.time() - start)))

test time :  04:14


In [16]:
zip_file_test = "test.zip"
output_folder_test = "data/"
start = time.time()
# Zip 파일 열기
with zipfile.ZipFile(zip_file_test, 'r') as zip_ref:
    # 모든 파일 압축 해제
    zip_ref.extractall(output_folder_test)
print('test time : ', time.strftime('%M:%S', time.gmtime(time.time() - start)))

os.remove(zip_file_test)

test time :  03:12


## 압축해제

In [7]:
import zipfile
import os
import uuid

def extract_zip_archive(zip_path, output_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

In [ ]:
# 입력 ZIP 파일 경로
zip_path = '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/data/train_crop_224by224.zip'
output_dir = '/content/data/train/'
extract_zip_archive(zip_path, output_dir)

In [8]:
# 입력 ZIP 파일 경로
zip_path = '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/data/test_crop_224by224.zip'
output_dir = '/content/data/test/'
extract_zip_archive(zip_path, output_dir)

In [6]:
# 입력 ZIP 파일 경로
zip_path = '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/data/train2_crop_224by224.zip'
output_dir = '/content/data/train/'
extract_zip_archive(zip_path, output_dir)

In [7]:
import os
import random

def shuffle_files(directory):
    file_list = os.listdir(directory)
    random.shuffle(file_list)

shuffle_files("/content/data/train/fake_images")
shuffle_files("/content/data/train/real_images")
#shuffle_files("/content/data/test/images")

In [20]:
# 디렉토리 별 파일 개수

def count_files(directory):
    file_count = sum(len(files) for _, _, files in os.walk(directory))
    print(f"Number of files in {directory}: {file_count}")

count_files("/content/data/train/fake_images") # 30283
count_files("/content/data/train/real_images") # 30584
count_files("/content/data/test/images") # 58348


Number of files in /content/data/train/fake_images: 56668
Number of files in /content/data/train/real_images: 62547
Number of files in /content/data/test/images: 0


In [19]:
# test 데이터셋으로 학습하기
import pandas as pd

test_df = pd.read_csv('/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/results/predict/eff_1_F_N2-6_data100%_K-Fold-5_20ep_20230706_091051/result.csv')
test_df.head()

,ImageId,answer
0,test_00000.png,1
1,test_00001.png,1
2,test_00002.png,1
3,test_00003.png,1
4,test_00004.png,0


In [20]:
# 1차적으로 학습 결과 토대로 테스트 데이터를 학습에 사용 (0.8068)
import os
import shutil

def move_images_to_directories(original_directory, fake_directory, real_directory):
    os.makedirs(fake_directory, exist_ok=True)
    os.makedirs(real_directory, exist_ok=True)

    # 원본 디렉토리 내 파일 목록 가져오기
    files = os.listdir(original_directory)

    for filename in files:
        # 파일 경로 생성
        filepath = os.path.join(original_directory, filename)
        if test_df[test_df['ImageId'] == f'{filename}']['answer'].values[0] == 1: # fake
            target_directory = fake_directory
        else:
            target_directory = real_directory

        # 파일을 타겟 디렉토리로 이동
        shutil.move(filepath, target_directory)


In [21]:
original_directory = "/content/data/test/images"
fake_directory = "/content/data/train/fake_images"
real_directory = "/content/data/train/real_images"

move_images_to_directories(original_directory, fake_directory, real_directory)


### crop+resize

In [23]:
import os
from PIL import Image

def resize_and_save_images(source_directory, target_directory, target_size):
    # 대상 디렉토리 생성
    os.makedirs(target_directory, exist_ok=True)

    # 디렉토리 내의 이미지 파일 목록 가져오기
    image_files = [file for file in os.listdir(source_directory) if file.lower().endswith((".png", ".jpg", ".jpeg", ".gif", ".bmp"))]

    # 이미지 파일마다 함수 적용
    for filename in image_files:
        source_path = os.path.join(source_directory, filename)
        target_path = os.path.join(target_directory, filename)

        # 이미지 열기
        image = Image.open(source_path)
        width, height = image.size

        # 가로, 세로 중 큰 쪽을 작은 쪽만큼 맞춰서 정사각형으로 맞추기
        if width > height:
            diff = (width - height) // 2
            image = image.crop((diff, 0, width - diff, height))
        else:
            diff = (height - width) // 2
            image = image.crop((0, diff, width, height - diff))

        # 리사이즈
        resized_image = image.resize(target_size, Image.ANTIALIAS)

        # 이미지 저장
        resized_image.save(target_path)
        #print(f"Image saved: {target_path}")qz


In [24]:
# Fake - Train
source_directory = "/content/data/train/fake_images"
target_directory = "/content/crop/data/train/fake_images"
target_size = (288, 288)
resize_and_save_images(source_directory, target_directory, target_size)

In [25]:
# Real - Train
source_directory = "/content/data/train/real_images"
target_directory = "/content/crop/data/train/real_images"
target_size = (288, 288)
resize_and_save_images(source_directory, target_directory, target_size)

## GPU 캐시 초기화

In [30]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

## Run(Train)

In [31]:
!python '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/train.py' \
--datadir /content/data/train \
--model effnet \
--n_epochs 1 \
--batch_size 16 \
--data_size 1.0 \
--num_workers 1 \
--lr 5.0e-3 \
--additional_learning True

{'datadir': '/content/data/train', 'model': 'effnet', 'n_epochs': 1, 'batch_size': 16, 'data_size': 1.0, 'num_workers': 1, 'lr': 0.005, 'additional_learning': True, 'optimizer': 'adam', 'loss': 'bce', 'metric': ['accuracy', 'f1macro'], 'early_stopping_target': 'val_accuracy', 'early_stopping_patience': 10, 'early_stopping_mode': 'max', 'amp': False, 'gpu': 0, 'seed': 42, 'val_size': 0.3, 'shuffle': True, 'pin_memory': True, 'drop_last': True, 'debug': False, 'wandb': False, 'logging_interval': 100, 'plot': ['loss', 'accuracy', 'f1macro', 'elapsed_time'], 'elapsed_time': True}
/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real
Directory '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/results/train/20230706_141021' is empty and has been deleted.
Fake : 18470 / Real : 18469
Train 0/1
100% 2308/2308 [13:57<00:00,  2.76it/s]
Val 0/1
100% 989/989 [05:27<00:00,  3.02it/s]


- eff_1_F_N2-6_data100%_K-Fold-5_20ep 학습
- test 셋 public ( 0.8065 )
- test 셋으로 기존 모델 추가 학습


- 궁금한점
  1. test셋 Public 성능은 0.8065인데, 내가 사전학습 시킨 모델로 test셋만 추가 학습시키면 0 에폭에서 ACC가 0.74가 나온다? 이게 무슨 의미일까?
  

- effnet_b0 + Pretrained = True + 20ep
  - 0.7173(pulic)
- effnet_b0 + Pretrained = False + 20ep + DropOut 1개
  - 0.7365(pulic)
- effnet_b0 + Pretrained = False + 35ep + DropOut 1개
  - 모름
- effnet_b0 + Pretrained = False + 20ep + L2 Norm + DropOut 2개
  - 학습안됨
- effnet_b0 + Pretrained = False + 20ep + L2 Norm + DropOut 1개
  - 학습안됨



## Run(Predict)

In [9]:
# test셋 데이터 이미지를 32로 먼저 바꾸기
!python '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/predict.py' \
--datadir /content/data/test/images \
--sample_submission_path '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/data/sample_submission.csv' \
--train_serial 'eff_1_F_N2-6_data100%_K-Fold-5_20ep' \
--batch_size 32

{'datadir': '/content/data/test/images', 'sample_submission_path': '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/data/sample_submission.csv', 'train_serial': 'eff_1_F_N2-6_data100%_K-Fold-5_20ep', 'batch_size': 32, 'gpu': 0, 'seed': 42}
100% 1824/1824 [07:14<00:00,  4.20it/s]
Done


## Run(Predict_Valid)

In [10]:
#@title
!python '/content/drive/MyDrive/github/Fake or Real 판별/Fake_or_Real/predict_valid.py' \
--datadir /content/data/train \
--train_serial 'eff_1_F_N2-6_data100%_K-Fold-5_20ep' \
--batch_size 32

{'datadir': '/content/data/train', 'train_serial': 'eff_1_F_N2-6_data100%_K-Fold-5_20ep', 'batch_size': 32, 'gpu': 0, 'seed': 42, 'val_size': 0.3}
Fake : 21198 / Real : 21198
['/content/data/train/real_images/real_02569.png', '/content/data/train/fake_images/0b0e38c5-7804-428c-be74-5c687390626a.png', '/content/data/train/fake_images/8b2e9577-6dee-4d1a-a2cd-7bac7b8507f4.png', '/content/data/train/real_images/5cccdeb5-35a1-46b2-827e-d53a1d130d36.png', '/content/data/train/fake_images/31b3fbe2-eeae-4eb5-8a41-d327a04f5f9c.png', '/content/data/train/fake_images/fake_02478.png', '/content/data/train/real_images/real_01291.png', '/content/data/train/real_images/c6abfc64-bf9e-4ce0-a4ac-1a2d67a4f93d.png', '/content/data/train/real_images/real_03100.png', '/content/data/train/fake_images/2d9efc91-bc16-4b7e-a5ac-87339526b8e5.png', '/content/data/train/fake_images/3b72e6f7-c3b1-46f7-9301-949200f48d67.png', '/content/data/train/real_images/image_193_0.png', '/content/data/train/real_images/93ae4d35